# Fine-tuning BERT for multi-label text classification

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DougTrajano/ToChiquinho/blob/toxicity_type_detection/notebooks/toxicity_type_detection_colab.ipynb) <- Edit it later

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way. 

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.

## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [ ]:
!pip install -q transformers datasets

In [1]:
from dataclasses import dataclass, field

@dataclass
class Parameters:
    max_seq_length: int = field(
        default=512,
        metadata={
            "help": (
                "The maximum total input sequence length after tokenization. Sequences longer "
                "than this will be truncated, sequences shorter will be padded."
            )
        }
    )
    
    model_name: str = field(
        default="neuralmind/bert-base-portuguese-cased",
        metadata={
            "help": "The name of the model to use. It must be a model name or a path to a directory containing model weights."
        }
    )
        
    model_type: str = field(
        default="bert",
        metadata={
            "help": "The type of the model to use. It must be a model type in the list of available models."
        }
    )

    num_train_epochs: int = field(
        default=1,
        metadata={
            "help": "The number of epochs to train the model. An epoch is an iteration over the entire training set."
        }
    )

    num_train_epochs_per_child: int = field(
        default=1,
        metadata={
            "help": "The number of epochs to train the model. An epoch is an iteration over the entire training set."
        }
    )

    batch_size: int = field(
        default=4,
        metadata={
            "help": "The batch size to use for training and evaluation."
        }
    )

    validation_split: float = field(
        default=0.2,
        metadata={
            "help": "The percentage of the training set to use as validation set."
        }
    )

    learning_rate: float = field(
        default=2e-5,
        metadata={
            "help": "The initial learning rate for Adam."
        }
    )

    seed: int = field(
        default=1993,
        metadata={
            "help": "The seed to use for random number generation."
        }
    )

params = Parameters()
params

Parameters(max_seq_length=512, model_name='neuralmind/bert-base-portuguese-cased', model_type='bert', num_train_epochs=1, num_train_epochs_per_child=1, batch_size=4, validation_split=0.2, learning_rate=2e-05, seed=1993)

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [3]:
import os
import json
import pandas as pd
from typing import List, Dict, Union
from datasets import Dataset
from kaggle.api.kaggle_api_extended import KaggleApi

def download_dataset(
        output_files: Union[str, List[str]] = "train.csv",
        dataset_files: List[str] = [
            "olidbr.csv.zip",
            "train.csv",
            "test.csv",
            "train_metadata.csv",
            "test_metadata.csv",
            "additional_data.json",
            "train.json",
            "test.json"
        ]) -> Dict[str, Union[Dict, pd.DataFrame]]:
    """Download dataset from Kaggle.

    Args:
    - output_files: List of files to be outputted.
    - dataset_files: List of files to be downloaded and deleted.

    Returns:
    - A dictionary with the output files as keys and the content as values.
    """

    # Download OLID-BR dataset
    for file in dataset_files:
        if not os.path.exists(file):
            print(f"Downloading OLID-BR from Kaggle.")
            kaggle = KaggleApi()
            kaggle.authenticate()
            kaggle.dataset_download_files(dataset="olidbr", unzip=True)

    # Load data
    result = {}
    for file in output_files:
        if file.endswith(".csv"):
            result[file] = pd.read_csv(file)
        elif file.endswith(".json"):
            result[file] = json.load(open(file, "r"))
        else:
            raise ValueError(f"File {file} is not supported.")

    # Delete files
    for file in dataset_files:
        if os.path.exists(file):
            os.remove(file)

    return result

dataset = download_dataset(["train.csv", "test.csv"])

train_df = dataset["train.csv"]
test_df = dataset["test.csv"]

del dataset

def preprocessing(df: pd.DataFrame) -> pd.DataFrame:
    """Preprocess the dataset.

    Args:
    - df: The dataset to be preprocessed.

    Returns:
    - The preprocessed dataset.
    """
    import warnings

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        
        # Filter only offensive comments
        df = df[df["is_offensive"] == "OFF"]

        # Remove religious_intolerance that has only one  sample
        if "religious_intolerance" in df.columns:
            df.drop("religious_intolerance", axis=1, inplace=True)

        # Filter only offensive comments with at least one toxicity label
        df = df.loc[df.select_dtypes("bool").sum(axis=1).ge(1)]

        return df.reset_index(drop=True)

train_df = preprocessing(train_df)
test_df = preprocessing(test_df)

In [4]:
train_df

,id,text,is_offensive,is_targeted,targeted_type,toxic_spans,health,ideology,insult,lgbtqphobia,other_lifestyle,physical_aspects,profanity_obscene,racism,sexism,xenophobia
0,c779826dc43f460cb18e8429ca443477,Pior do que adolescentezinhas de merda...são p...,OFF,UNT,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",False,False,True,False,False,False,True,False,True,False
1,e64148caa4474fc79298e01d0dda8f5e,USER Toma no cu é vitamina como tu e tua prima.,OFF,TIN,GRP,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",False,False,True,False,False,False,True,False,False,False
2,cc66b54eeec24607a67e2259134a1cdd,"Muito bom, pena a circunstâncias serem ruins, ...",OFF,UNT,NaN,"[119, 120, 121, 122, 123, 124, 125, 126, 127, ...",False,False,True,False,False,False,False,False,False,False
3,a3d7839456ae4258a70298fcf637952e,"Podia ter beijo também, pra ver se o homofóbic...",OFF,UNT,NaN,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",False,False,True,False,False,False,False,False,False,False
4,b830374760cc44789f1493c6b228ebc1,"o monark não é racista que filha da putagem, j...",OFF,TIN,GRP,"[36, 37, 38, 39, 40, 41, 42]",False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4267,4ca2cbe2ea284e6a8e713f6a472790a3,"USER do caralho, só não da pra add no orkut po...",OFF,TIN,NaN,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 86, 87, 88, ...",False,False,True,False,False,False,True,False,False,False
4268,a986f6f63d8240c387c57f64ed287ab8,"pô, que propaganda bonita. povo besta, viu.",OFF,UNT,NaN,"[32, 33, 34, 35, 36]",False,False,True,False,False,False,False,False,False,False
4269,31b64791facf43a2b27139dfa187740f,Olha que legal agora estamos chegando ao final...,OFF,UNT,NaN,[],False,False,True,True,False,False,False,False,False,False
4270,d00cca68919e45c2a76309ce8c9d55e8,"foram embora só agora da minha casa, bando de ...",OFF,TIN,GRP,"[45, 46, 47, 48, 49, 50, 51, 52, 53, 54]",False,False,True,False,False,False,True,False,False,False


In [8]:
columns = [
    "text",
    "health",
    "ideology",
    "insult",
    "lgbtqphobia",
    "other_lifestyle",
    "physical_aspects",
    "profanity_obscene",
    "racism",
    "sexism",
    "xenophobia"
]

dataset = Dataset.from_dict(
    {
        k: v for k, v in train_df[columns].to_dict(orient="list").items()
    }
)

dataset = dataset.train_test_split(
    test_size=params.validation_split,
    shuffle=True,
    seed=params.seed
)

dataset["validation"] = dataset.pop("test")

# Add test dataset
dataset["test"] = Dataset.from_dict(
    {
        k: v for k, v in test_df[columns].to_dict(orient="list").items()
    }
)


As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'health', 'ideology', 'insult', 'lgbtqphobia', 'other_lifestyle', 'physical_aspects', 'profanity_obscene', 'racism', 'sexism', 'xenophobia'],
        num_rows: 3417
    })
    validation: Dataset({
        features: ['text', 'health', 'ideology', 'insult', 'lgbtqphobia', 'other_lifestyle', 'physical_aspects', 'profanity_obscene', 'racism', 'sexism', 'xenophobia'],
        num_rows: 855
    })
    test: Dataset({
        features: ['text', 'health', 'ideology', 'insult', 'lgbtqphobia', 'other_lifestyle', 'physical_aspects', 'profanity_obscene', 'racism', 'sexism', 'xenophobia'],
        num_rows: 1438
    })
})

Let's check the first example of the training split:

In [11]:
example = dataset["train"][0]
example

{'text': 'Ela mendigava o amor💘, na falta de atenção, o mendigo não negou!!!!',
 'health': False,
 'ideology': False,
 'insult': True,
 'lgbtqphobia': False,
 'other_lifestyle': False,
 'physical_aspects': False,
 'profanity_obscene': False,
 'racism': False,
 'sexism': False,
 'xenophobia': False}

The dataset consists of tweets, labeled with one or more emotions. 

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [12]:
labels = [label for label in dataset['train'].features.keys() if label not in ["text"]]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['health',
 'ideology',
 'insult',
 'lgbtqphobia',
 'other_lifestyle',
 'physical_aspects',
 'profanity_obscene',
 'racism',
 'sexism',
 'xenophobia']

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [13]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

def preprocess_data(examples, tokenizer, max_seq_length):
    # take a batch of texts
    text = examples["text"]

    # encode them
    encoding = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_seq_length
    )

    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}

    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))

    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding

In [14]:
encoded_dataset = dataset.map(
    preprocess_data,
    batched=True,
    remove_columns=dataset["train"].column_names,
    fn_kwargs={
        "tokenizer": tokenizer,
        "max_seq_length": params.max_seq_length
    }
)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [15]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [16]:
tokenizer.decode(example['input_ids'])

'[CLS] Ela mendigava o [UNK], na falta de atenção, o mendigo não negou!!!! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [17]:
example['labels']

[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [18]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['insult']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [19]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "neuralmind/bert-base-portuguese-cased", 
    problem_type="multi_label_classification",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [ ]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [ ]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  1523,  4737,  2003,  1037,  2091,  7909,  2006,  1037,  3291,
         2017,  2089,  2196,  2031,  1005,  1012, 11830, 11527,  1012,  1001,
        14354,  1001,  4105,  1001,  4737,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [ ]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput([('loss',
                           tensor(0.6442, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)),
                          ('logits',
                           tensor([[-0.2136,  0.1645, -0.2064, -0.0940, -0.5121,  0.0718,  0.1167, -0.1767,
                                     0.0968,  0.0115, -0.1126]], grad_fn=<AddmmBackward0>))])

Let's start training!

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 6838
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4275


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.279300,0.315630,0.690058,0.786618,0.262980
2,0.237500,0.308757,0.702635,0.795974,0.273138
3,0.202900,0.319157,0.709615,0.805806,0.279910
4,0.180800,0.324087,0.702689,0.798627,0.277652
5,0.157700,0.329103,0.701452,0.798265,0.272009


***** Running Evaluation *****
  Num examples = 886
  Batch size = 8
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-855
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-855/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-855/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-855/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-855/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 886
  Batch size = 8
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-1710
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-1710/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-1710/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-1710/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoin

TrainOutput(global_step=4275, training_loss=0.20808937385068302, metrics={'train_runtime': 676.8489, 'train_samples_per_second': 50.513, 'train_steps_per_second': 6.316, 'total_flos': 2249123476753920.0, 'train_loss': 0.20808937385068302, 'epoch': 5.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 886
  Batch size = 8


{'epoch': 5.0,
 'eval_accuracy': 0.2799097065462754,
 'eval_f1': 0.7096149188665537,
 'eval_loss': 0.31915703415870667,
 'eval_roc_auc': 0.805805895058838,
 'eval_runtime': 4.7187,
 'eval_samples_per_second': 187.766,
 'eval_steps_per_second': 23.524}

## Inference

Let's test the model on a new sentence:

In [ ]:
text = "I'm happy I can finally train a model for multi-label classification"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [ ]:
logits = outputs.logits
logits.shape

torch.Size([1, 11])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['joy', 'optimism']
